In [ ]:
import param
import panel as pn

from bokeh.sampledata.iris import flowers
from bokeh.sampledata.autompg import autompg_clean
from bokeh.sampledata.population import data

pn.extension()

This example demonstrates Panel's reactive programming paradigm using the Param library to express parameters, plus methods with computation depending on those parameters. This pattern can be used to update the displayed views whenever a parameter value changes, without re-running computation unnecessarily.

In [ ]:
class ReactiveTables(param.Parameterized):
    
    title = param.String(default='Data Summary')
    
    dataset = param.ObjectSelector(default='iris', objects=['iris', 'autompg', 'population'])
    
    rows = param.Integer(default=10, bounds=(0, 100))
    
    @param.depends('dataset')
    def data(self):
        if self.dataset == 'iris':
            return flowers
        elif self.dataset == 'autompg':
            return autompg_clean
        else:
            return data

    @param.depends('data')
    def summary(self):
        return self.data().describe()
    
    @param.depends('title')
    def header(self):
        return '##' + self.title
    
    @param.depends('data', 'rows')
    def table(self):
        return self.data().iloc[:self.rows]
    
    def panel(self):
        return pn.Row(
            self.param,
            pn.Column(self.header, self.summary, self.table),
            min_height=1000)
    
reactive = ReactiveTables()
reactive.panel().servable()